In [2]:
!pip install split-folders
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import splitfolders

In [3]:
!unzip dataset.zip
splitfolders.ratio(
    'dataset',
    output='train_val_test',
    seed=42,
    ratio=(0.7, 0.15, 0.15),
    group_prefix=None,
    move=False
)

Archive:  dataset.zip
replace dataset/Bear/bear_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/Bear/bear_1.jpg  
  inflating: dataset/Bear/Bear_10.jpg  
  inflating: dataset/Bear/Bear_10_1.jpg  
  inflating: dataset/Bear/Bear_10_2.jpg  
  inflating: dataset/Bear/Bear_10_3.jpg  
  inflating: dataset/Bear/Bear_11.jpeg  
  inflating: dataset/Bear/Bear_11_1.jpg  
  inflating: dataset/Bear/Bear_11_2.jpg  
  inflating: dataset/Bear/Bear_11_3.jpg  
  inflating: dataset/Bear/Bear_12.jpeg  
  inflating: dataset/Bear/Bear_12_1.jpg  
  inflating: dataset/Bear/Bear_12_2.jpg  
  inflating: dataset/Bear/Bear_12_3.jpg  
  inflating: dataset/Bear/Bear_13.jpeg  
  inflating: dataset/Bear/Bear_13_1.jpg  
  inflating: dataset/Bear/Bear_13_2.jpg  
  inflating: dataset/Bear/Bear_13_3.jpg  
  inflating: dataset/Bear/Bear_13_4.jpg  
  inflating: dataset/Bear/Bear_14.jpeg  
  inflating: dataset/Bear/Bear_14_1.jpg  
  inflating: dataset/Bear/Bear_14_2.jpg  
  inflating: dataset/Bear/Bear_1

Copying files: 1932 files [00:00, 6399.07 files/s]


In [4]:
from keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:
train_generator = train_datagen.flow_from_directory(
    'train_val_test/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'train_val_test/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'train_val_test/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1345 images belonging to 15 classes.
Found 282 images belonging to 15 classes.
Found 305 images belonging to 15 classes.


In [6]:
import json
with open('class_names.json', 'w') as f:
    json.dump(test_generator.class_indices, f)

In [7]:
from keras.applications import EfficientNetB0
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

In [8]:
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
inputs = base_model.input
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
outputs = Dense(15, activation='softmax')(x)

model = Model(inputs, outputs)

In [10]:
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    class_weight=class_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 18s 435ms/step - accuracy: 0.9379 - loss: 0.5314 - val_accuracy: 0.9883 - val_loss: 0.3808
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.4259 - val_accuracy: 0.9961 - val_loss: 0.3771
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 41s 946ms/step - accuracy: 0.9344 - loss: 0.5457 - val_accuracy: 0.9961 - val_loss: 0.3709
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9688 - loss: 0.4691 - val_accuracy: 0.9961 - val_loss: 0.3725
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 18s 427ms/step - accuracy: 0.9438 - loss: 0.5109 - val_accuracy: 0.9844 - val_loss: 0.3616
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.4016 - val_accuracy: 0.9883 - val_loss: 0.3566
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 21s 468ms/step - accuracy: 0.9370 - loss: 0.5167 - val_accuracy: 0.9883 - val_loss: 0.3527
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.4349 - val_accuracy: 0.

In [15]:
# Evaluate
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // 32)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9848 - loss: 0.3094
Test Accuracy: 98.96%


In [16]:
import numpy as np
preds = model.predict(test_generator, steps=test_generator.samples // 32 + 1)
pred_classes = np.argmax(preds, axis=1)
true_classes = test_generator.classes
from sklearn.metrics import classification_report
print(classification_report(true_classes, pred_classes, target_names=test_generator.class_indices.keys()))

10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 946ms/step
              precision    recall  f1-score   support

        Bear       1.00      0.95      0.97        20
        Bird       1.00      1.00      1.00        22
         Cat       1.00      1.00      1.00        19
         Cow       1.00      0.95      0.98        21
        Deer       1.00      1.00      1.00        20
         Dog       1.00      1.00      1.00        19
     Dolphin       1.00      1.00      1.00        20
    Elephant       0.91      1.00      0.95        21
     Giraffe       1.00      1.00      1.00        20
       Horse       1.00      1.00      1.00        20
    Kangaroo       1.00      0.95      0.97        20
        Lion       1.00      1.00      1.00        21
       Panda       0.95      1.00      0.98        21
       Tiger       1.00      1.00      1.00        20
       Zebra       1.00      1.00      1.00        21

    accuracy                           0.99       305
   macro avg       0.99      0.99     

In [17]:
model.save("animal_classifier_model.keras")

In [18]:
from google.colab import files
files.download("animal_classifier_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>